In [1]:
import os
import json
import pandas as pd
from pyBioInfo.IO.File import GtfFile, GtfTranscriptBuilder
from pyBioInfo.Utils import BlockTools

# Make introns.bed.gz

In [22]:
f_gtf = "/home/chenzonggui/species/homo_sapiens/GRCh38.p13/gencode.v39.annotation.sorted.gtf.gz"

with GtfFile(f_gtf) as f:
    records = [x for x in f]
transcripts = list(GtfTranscriptBuilder(records))

introns = []
for t in transcripts:
    for start, end in BlockTools.gaps(t.blocks):
        introns.append((t.chrom, start, end, "Intron", ".", t.strand))
print("All introns:", len(introns))

introns = set(introns)
print("Uniq introns:", len(introns))

with open("results/introns/introns.bed", "w+") as fw:
    for row in sorted(introns):
        fw.write("\t".join(map(str, row)) + "\n")
        
cmd = "bgzip -f results/introns/introns.bed"
assert os.system(cmd) == 0
cmd = "tabix -p bed -f results/introns/introns.bed.gz"
assert os.system(cmd) == 0

All introns: 1307815
Uniq introns: 395195


# Normalized intron quatification

In [29]:
dates = ["20221128", "20221205"]

samples = ["0h-1", "0h-2", "3h-1", "3h-2", "6h-1", "6h-2"]

for date in dates:

    factors = json.load(open("results/halflife/conversion_factors.%s.json" % date))

    paths = [
        "results/introns/counts/%s_K562_Actd_0h_rep1.tsv" % date,
        "results/introns/counts/%s_K562_Actd_0h_rep2.tsv" % date,
        "results/introns/counts/%s_K562_Actd_3h_rep1.tsv" % date,
        "results/introns/counts/%s_K562_Actd_3h_rep2.tsv" % date,
        "results/introns/counts/%s_K562_Actd_6h_rep1.tsv" % date,
        "results/introns/counts/%s_K562_Actd_6h_rep2.tsv" % date,
    ]
    
    array = []
    for sample, path in zip(samples, paths):
        d = pd.read_csv(path, sep="\t", header=None)
        d.columns = ["Chrom", "Start", "End", "Name", "Count", "Strand"]
        s = d["Count"]
        s.name = sample
        array.append(s)
    dat = pd.concat(array, axis=1)
    dat = pd.concat([d[["Chrom", "Start", "End", "Strand"]], dat], axis=1)

    dat.index = ["%s_%s_%s" % (chrom, start, end) for chrom, start, end in dat[["Chrom", "Start", "End"]].values]
    dat.index.name = "ID"

    for sample in samples:
        x = factors["%s,%s" % (sample, "0h-1")]
        dat["%s_adj" % sample] = dat[sample] * x
    for sample in samples:
        dat["%s_adj_p" % sample] = dat["%s_adj" % sample] / dat[["0h-1_adj", "0h-2_adj"]].mean(axis=1)
        
    dat.to_csv("results/introns/normalized_counts.%s.tsv" % date, sep="\t")